In [1]:
import pytest
import numpy as np
from modules.utils.services import embedder_services

# ==========================================
# 🧩 1️⃣ Kiểm tra Dense Embedding
# ==========================================
def test_encode_dense_basic():
    text = "Phân tích thị trường chứng khoán Việt Nam"
    dense_vecs = embedder_services.encode_dense(text)
    assert isinstance(dense_vecs, list), "Dense output phải là list"
    assert isinstance(dense_vecs[0], list), "Mỗi vector phải là list"
    assert len(dense_vecs[0]) == 384, f"Chiều dense vector phải = 384, hiện tại = {len(dense_vecs[0])}"
    print("\n✅ [Dense] OK - Chiều vector:", len(dense_vecs[0]))
    print("🔹 First 5 dims:", dense_vecs[0][:5])


# ==========================================
# 🧩 2️⃣ Kiểm tra Fit BM25
# ==========================================
def test_fit_bm25_and_vocab():
    corpus = [
        "Vingroup hỗ trợ đồng bào miền Trung",
        "Thị trường chứng khoán Việt Nam biến động",
        "Giá cổ phiếu VCB tăng mạnh hôm nay",
        "Techcombank và Vietcombank công bố kết quả kinh doanh quý 3"
    ]

    embedder_services.fit_bm25(corpus)
    assert embedder_services.bm25 is not None, "BM25 chưa được fit"
    assert len(embedder_services.vocab) > 0, "Từ vựng BM25 rỗng"
    print("\n✅ [BM25] OK - Vocab size:", len(embedder_services.vocab))


# ==========================================
# 🧩 3️⃣ Kiểm tra Sparse Embedding
# ==========================================
def test_encode_sparse_after_fit():
    text = "Phân tích cổ phiếu VCB"
    sparse_vecs = embedder_services.encode_sparse(text)
    assert isinstance(sparse_vecs, list), "Sparse output phải là list"
    assert "indices" in sparse_vecs[0] and "values" in sparse_vecs[0], "Cấu trúc sparse không hợp lệ"
    print("\n✅ [Sparse] OK - Số token trong vector:", len(sparse_vecs[0]['indices']))
    print("🔹 Sample indices:", sparse_vecs[0]['indices'][:5])
    print("🔹 Sample values:", sparse_vecs[0]['values'][:5])


# ==========================================
# 🧩 4️⃣ Kiểm tra Dense + Sparse song song
# ==========================================
def test_hybrid_encode():
    text = "Giá cổ phiếu VCB tăng mạnh"
    dense = embedder_services.encode_dense(text)
    sparse = embedder_services.encode_sparse(text)

    assert len(dense[0]) == 384, "Dense vector phải có 384 chiều"
    assert isinstance(sparse[0]['indices'], list), "Sparse indices phải là list"
    print("\n✅ [Hybrid] OK - Dense(384) + Sparse(length=", len(sparse[0]['indices']), ")")
    print("🔹 Dense first dims:", dense[0][:5])


# ==========================================
# 🧩 5️⃣ Kiểm tra lỗi nếu chưa fit BM25
# ==========================================
def test_sparse_before_fit():
    # Tạo instance mới chưa fit để test
    from modules.utils.services import EmbedderServices
    new_embedder = EmbedderServices(auto_fit=False)
    with pytest.raises(ValueError, match="BM25 chưa được fit"):
        new_embedder.encode_sparse("cổ phiếu VCB")
    print("\n✅ [Error Handling] OK - Cảnh báo chưa fit BM25 hoạt động đúng")
if __name__ == "__main__":
    test_encode_dense_basic()
    test_fit_bm25_and_vocab()
    test_encode_sparse_after_fit()
    test_hybrid_encode()
    test_sparse_before_fit()

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1015 08:51:00.512000 1456 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.
EmbedderServices device: cuda
RerankerServices device: cuda

✅ [Dense] OK - Chiều vector: 384
🔹 First 5 dims: [-0.42621153593063354, 0.4696739912033081, -0.08442647755146027, -0.06147570535540581, -0.23391616344451904]

✅ [BM25] OK - Vocab size: 34

✅ [Sparse] OK - Số token trong vector: 3
🔹 Sample indices: [6, 15, 25]
🔹 Sample values: [0.8472978603872037, 0.8472978603872037, 0.8472978603872037]

✅ [Hybrid] OK - Dense(384) + Sparse(length= 6 )
🔹 Dense first dims: [-0.3635924756526947, 0.16256432235240936, -0.22329771518707275, 0.07123153656721115, -0.05664300546050072]
EmbedderServices device: cuda

✅ [Error Handling] OK - Cảnh báo chưa fit BM25 hoạt động đúng


In [1]:
from modules.core.state import GlobalState
from modules.nodes.embedder import embed_query
import numpy as np

def test_embedder():
    print("=== 🧩 TEST EMBEDDER MODULE (rag & hybrid) ===")

    for route_type in ["rag", "hybrid"]:
        print(f"\n🔹 TEST ROUTE: {route_type.upper()}")
        state = GlobalState(user_query="Hôm nay có tin tức gì mới?", route_to=route_type)

        state = embed_query(state)

        if state.query_embedding is None:
            print("❌ Không tạo được embedding.")
            print("Debug info:", state.debug_info)
            continue

        dense_vec = state.query_embedding.get("dense_vector")
        sparse_vec = state.query_embedding.get("sparse_vector")

        print("✅ Embedding thành công.")
        print(f"🧠 Dense vector length: {len(dense_vec) if dense_vec is not None else 'None'}")
        if isinstance(dense_vec, (list, np.ndarray)):
            print(f"🔹 First 5 dims: {dense_vec[:5]}")
        else:
            print("⚠️ Dense vector không phải list hoặc np.ndarray")

        print(f"🌾 Sparse vector: {'OK' if sparse_vec else 'None'}")
        print(f"📜 Debug Info: {state.debug_info}")
        print("LLM status:", state.llm_status)
        print("Embed route:", state.debug_info.get('embed_route'))

if __name__ == "__main__":
    test_embedder()


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1025 19:25:10.871000 16968 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.
EmbedderServices device: cuda
RerankerServices device: cuda


Device set to use cuda:0
c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


=== 🧩 TEST EMBEDDER MODULE (rag & hybrid) ===

🔹 TEST ROUTE: RAG
✅ Embedding thành công.
🧠 Dense vector length: 384
🔹 First 5 dims: [-0.6698017120361328, 0.020612990483641624, -0.15784049034118652, 0.1114264577627182, -0.6839015483856201]
🌾 Sparse vector: OK
📜 Debug Info: {'embed_dense_dim': 384, 'embed_sparse_status': 'ok', 'embed_status': 'success', 'embed_query_text': 'Hôm nay có tin tức gì mới?', 'embed_route': 'rag'}
LLM status: embed_success
Embed route: rag

🔹 TEST ROUTE: HYBRID
✅ Embedding thành công.
🧠 Dense vector length: 384
🔹 First 5 dims: [-0.6698017120361328, 0.020612990483641624, -0.15784049034118652, 0.1114264577627182, -0.6839015483856201]
🌾 Sparse vector: OK
📜 Debug Info: {'embed_dense_dim': 384, 'embed_sparse_status': 'ok', 'embed_status': 'success', 'embed_query_text': 'Hôm nay có tin tức gì mới?', 'embed_route': 'hybrid'}
LLM status: embed_success
Embed route: hybrid


In [1]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query   # 🧠 Gọi Processor để detect intent + time_filter
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from datetime import datetime
import pytz

def test_vector_db_realdata():
    """
    🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF)
    ------------------------------------------------
    ✅ Gọi Processor để detect intent + time_filter
    ✅ Kiểm tra embedding + hybrid search + reranker
    ✅ In chi tiết debug + khoảng thời gian lọc
    """
    print("\n=== 🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF) ===")

    query = "Hôm nay có tin tức gì mới?"
    state = GlobalState(user_query=query, debug=True)

    # --- Bước 1: Processor (detect intent + time_filter + tickers) ---
    state = processor_query(state)

    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    print(f"\n🧠 Query gốc: {state.user_query}")
    print(f"🗣️ Intent: {getattr(state, 'intent', 'unknown')}")
    print(f"⏰ Time Filter: {state.time_filter}")
    if state.time_filter:
        start_ts, end_ts = state.time_filter
        print(f"   → {datetime.fromtimestamp(start_ts, vn_tz)} → {datetime.fromtimestamp(end_ts, vn_tz)}")
    else:
        print("   → Không phát hiện mốc thời gian, sẽ dùng mặc định 3 ngày gần nhất.")

    # --- Chạy test cho 2 route: RAG và HYBRID ---
    for route in ["rag", "hybrid"]:
        print(f"\n🔹 TEST ROUTE: {route.upper()}")
        state.route_to = route

        try:
            # --- Bước 2: EMBEDDING ---
            state = embed_query(state)
            if not state.query_embedding:
                print("❌ Không tạo được embedding.")
                print("Debug info:", getattr(state, "debug_info", {}))
                continue

            dense_dim = len(state.query_embedding.get("dense_vector", []))
            print(f"✅ [Dense] OK - Chiều vector: {dense_dim}")

            sparse_vec = state.query_embedding.get("sparse_vector")
            if sparse_vec:
                print(f"✅ [Sparse] OK - Số token trong vector: {len(sparse_vec[0]['indices'])}")
            else:
                print("⚠️ [Sparse] Không có sparse vector (BM25 chưa fit hoặc corpus rỗng).")

            # --- Bước 3: SEARCH (Hybrid + Reranker) ---
            state = search_vector_db(state, top_k=5, alpha=0.7)

            # --- Kiểm tra reranker ---
            rerank_status = getattr(state, "debug_info", {}).get("reranker_status", "unknown")
            if "applied" in rerank_status:
                print(f"✅ [Reranker] ĐÃ ÁP DỤNG — Model: {state.debug_info.get('reranker_model')} | Device: {state.debug_info.get('reranker_device')}")
            elif "failed" in rerank_status:
                print(f"❌ [Reranker] LỖI: {rerank_status}")
            else:
                print(f"⚠️ [Reranker] BỎ QUA — {rerank_status}")

            # --- Bước 4: In kết quả ---
            print(f"\n=== 🔍 KẾT QUẢ TÌM KIẾM CHO TRUY VẤN: “{query}” ===")
            if not state.search_results:
                print("⚠️ Không tìm thấy kết quả nào trong Qdrant.")
            else:
                for idx, doc in enumerate(state.search_results, 1):
                    title = doc.get("title", "❌ Không có tiêu đề")
                    time = doc.get("time", "❌ Không rõ thời gian")
                    snippet = doc.get("content", "") or "(Không có nội dung trích dẫn)"
                    print(f"{idx}. [{doc.get('score', 0):.4f}] {title} — {time}")
                    print(f"   {snippet}\n")

            # --- Debug tổng kết ---
            print("📜 Debug Info:", getattr(state, "debug_info", {}))
            print("🧠 LLM Status:", getattr(state, "llm_status", "undefined"))

        except Exception as e:
            print(f"❌ Lỗi trong route {route}: {e}")

    print("\n✅ [Test] Hoàn tất kiểm thử Vector DB + Reranker.\n")


if __name__ == "__main__":
    test_vector_db_realdata()


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\vnai\scope\profile.py:562: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
W1027 10:22:41.299000 4408 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.
EmbedderServices device: cuda
RerankerServices device: cuda


Device set to use cuda:0
`return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.



=== 🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF) ===

🧠 Query gốc: Hôm nay có tin tức gì mới?
🗣️ Intent: rag
⏰ Time Filter: (1761498000, 1761584399)
   → 2025-10-27 00:00:00+07:00 → 2025-10-27 23:59:59+07:00

🔹 TEST ROUTE: RAG
✅ [Dense] OK - Chiều vector: 384
✅ [Sparse] OK - Số token trong vector: 7
✅ [Reranker] ĐÃ ÁP DỤNG — Model: None | Device: None

=== 🔍 KẾT QUẢ TÌM KIẾM CHO TRUY VẤN: “Hôm nay có tin tức gì mới?” ===
1. [0.6000] Có tin báo cáo tài chính doanh nghiệp, chứng khoán sẽ 'thoát hiểm'? — 27-10-2025 00:07:00
   vọng lợi nhuận quý III tích cực. Chúng tôi giữ quan điểm thận trọng, lựa chọn cơ hội đầu tư mới dựa trên vùng định giá hợp lý và kết quả kinh doanh thực chất của doanh nghiệp , chuyên gia khuyến nghị, Theo chuyên gia từ Chứng khoán Pinetree, trong ngắn hạn, thị trường sẽ thiên về xu hướng tích cực hơn là tiêu cực, với khả năng cao sẽ tiếp tục duy trì sideway đi lên trong biên độ hẹp. Dù vậy, xu hướng tăng mạnh sẽ khó xảy ra ngay, do dòng tiền vẫn chưa có sự lan tỏa t

In [2]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.router import route_intent
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.api import stock_api
from datetime import datetime
import pytz


def test_rag_pipeline_realdata():
    """
    🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT (CAFÉF)
    -------------------------------------------------------
    Chuỗi xử lý:
    Processor → Router → Embedder → VectorDB → Retriever
    - Tự động phát hiện intent và time_filter
    - In log chi tiết, giúp debug từng giai đoạn
    """

    print("\n=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===")

    # 1️⃣ Khởi tạo GlobalState
    state = GlobalState(
        user_query="Hôm nay có tin tức gì mới?",
        debug=True
    )
    print(f"\n🧠 [Input Query] {state.user_query}")

    # 2️⃣ Processor — xử lý truy vấn người dùng
    state = processor_query(state)
    print(f"✅ [Processor] processed_query = {state.processed_query}")
    print(f"   intent = {state.intent}, time_filter = {state.time_filter}")
    print(f"   tickers = {state.tickers}")

    # 3️⃣ Router — xác định hướng xử lý (api, rag, hybrid)
    state = route_intent(state)
    print(f"➡️ [Router] route_to = {state.route_to}, api_type = {getattr(state, 'api_type', None)}")

    # 4️⃣ Nếu query là dạng API (chứng khoán, thời tiết, v.v.)
    if getattr(state, "api_type", None) == "stock":
        symbol = state.tickers[0] if state.tickers else "VCB"
        state.api_response = stock_api.format_stock_info(symbol)
        print(f"📊 [API] Stock response for {symbol}: {state.api_response}")
        print("\n✅ [Pipeline Completed via API route]\n")
        return

    # 5️⃣ Embedder — tạo dense + sparse embedding
    state = embed_query(state)
    if not getattr(state, "query_embedding", None):
        print("❌ [Embedder] Không tạo được embedding.")
        return

    dense_dim = len(state.query_embedding.get("dense_vector", []))
    sparse_vector = state.query_embedding.get("sparse_vector")
    sparse_len = len(sparse_vector[0]["indices"]) if sparse_vector else 0
    print(f"✅ [Embedder] Dense dim = {dense_dim}, Sparse tokens = {sparse_len}")

    # 6️⃣ Vector DB — tìm kiếm từ Qdrant (Hybrid dense + sparse)
    state = search_vector_db(state, top_k=5, alpha=0.7)
    print(f"🔍 [VectorDB] Found {len(state.search_results)} results | llm_status={state.llm_status}")

    if not getattr(state, "search_results", None):
        print("⚠️ [VectorDB] Không có tài liệu nào trong Qdrant.")
        return

    # 7️⃣ Retriever — hợp nhất chunk, gắn nhãn thời gian, tạo context
    state = retrieve_documents(state, max_chars=2000)
    print(f"📄 [Retriever] Retrieved {len(state.retrieved_docs)} docs, context_len={len(state.context)}")

    # Hiển thị trước một phần context (tối đa 400 ký tự)
    preview = state.context[:400].replace("\n", " ")
    print(f"🧩 [Context Preview] {preview}...")

    # 8️⃣ Tổng kết
    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    now = datetime.now(vn_tz).strftime("%d-%m-%Y %H:%M:%S")
    print(f"\n🕒 [Run completed at] {now}")
    print(f"🧠 [LLM Status] {state.llm_status}")
    print("✅ [Test] Hoàn tất kiểm thử full RAG pipeline.\n")


if __name__ == "__main__":
    test_rag_pipeline_realdata()



=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===

🧠 [Input Query] Hôm nay có tin tức gì mới?
✅ [Processor] processed_query = hôm nay có tin tức gì mới
   intent = rag, time_filter = (1761498000, 1761584399)
   tickers = []
➡️ [Router] route_to = rag, api_type = None
✅ [Embedder] Dense dim = 384, Sparse tokens = 7
🔍 [VectorDB] Found 5 results | llm_status=vector_db_success
📄 [Retriever] Retrieved 2 docs, context_len=2261
🧩 [Context Preview] [Tin tức cập nhật đến: Hôm nay là Thứ hai, Ngày 27 tháng 10 năm 2025. Ngày hôm qua là Ngày 26 tháng 10 năm 2025. Ngày mai là Ngày 28 tháng 10 năm 2025. Tuần sau sẽ bắt đầu từ Ngày 03 tháng 11 năm 2025. Tuần trước bắt đầu từ Ngày 20 tháng 10 năm 2025.]  [Lịch sự kiện và tin vắn chứng khoán ngày 27/10/2025 | 27-10-2025 05:00:00] (score=0.625, rerank=-2.8142) tư và Xây dựng Thủy lợi Lâm Đồng: Kế toán...

🕒 [Run completed at] 27-10-2025 10:23:33
🧠 [LLM Status] retriever_success
✅ [Test] Hoàn tất kiểm thử full RAG pipeline.



In [3]:
from modules.utils.services import qdrant_services

points, _ = qdrant_services.client.scroll(
    collection_name="cafef_articles", 
    with_payload=True, 
    limit=5
)
for p in points:
    print(p.payload)


{'id': '0015a8ef4bd041daf8af9acd1247785f', 'title': 'Góc nhìn chuyên gia: Dòng tiền lớn nhập cuộc, VN-Index vẫn còn tiềm năng tăng thêm 20–30%', 'time': '12-10-2025 17:10:00', 'time_ts': 1760263800, 'summary': 'Chuyên gia khuyến nghị nhà đầu tư không nên mua đuổi bằng mọi giá, mà nên áp dụng\xa0chiến lược giải ngân theo từng phần.', 'url': 'https://cafef.vn/goc-nhin-chuyen-gia-dong-tien-lon-nhap-cuoc-vn-index-van-con-tiem-nang-tang-them-2030-188251012163930025.chn', 'content': 'nâng hạng. Đây sẽ là tín hiệu khởi đầu cho một giai đoạn mới của thị trường chứng khoán Việt Nam ,ông nhận định. Theo ông Hoàng, độ bền của xu hướng tăng sẽ phụ thuộc vào sự cộng hưởng của ba trụ cột: Thanh khoản – Lợi nhuận doanh nghiệp – Yếu tố vĩ mô. Thị trường hiện đang ở giai đoạn hai của chu kỳ tăng – giai đoạn bứt phá ( Public Participation theo Lý thuyết Dow). Trong giai đoạn này, thanh khoản dồi dào là yếu tố tiên quyết. Dòng vốn ngoại theo câu chuyện nâng hạng sẽ tiếp tục là động lực chính trong 6–12 t

In [8]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.router import route_intent
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.nodes.prompt_builder import build_prompt
from modules.api import stock_api
from datetime import datetime
import pytz

def test_prompt_realdata():
    """
    🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT
    ------------------------------------------------
    🔹 Pipeline: Processor → Router → Embedder → VectorDB → Retriever → Prompt
    🔹 Mục tiêu: kiểm tra prompt có đủ context, time_filter, API, và conversation history
    """
    print("\n=== 🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT ===\n")

    # 1️⃣ Khởi tạo state
    state = GlobalState()
    state.debug = True
    state.user_query = "Hôm nay có tin tức gì mới?"

    # 2️⃣ Processor
    state = processor_query(state)
    print(f"🧠 [Processor] processed_query = {state.processed_query}")
    if getattr(state, "time_filter", None):
        start_ts, end_ts = state.time_filter
        vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
        print(f"⏰ [TimeFilter] {datetime.fromtimestamp(start_ts, vn_tz)} → {datetime.fromtimestamp(end_ts, vn_tz)}")

    # 3️⃣ Router
    state = route_intent(state)
    print(f"➡️ [Router] route_to = {state.route_to}, api_type = {state.api_type}")

    # 4️⃣ API nếu cần
    if getattr(state, "api_type", None) == "stock":
        state.api_response = stock_api.format_stock_info("TCB")
        print(f"📊 [API] response = {state.api_response[:120]}...")

    # 5️⃣ Embedder
    state = embed_query(state)
    dense_dim = len(state.query_embedding.get("dense_vector", [])) if state.query_embedding else 0
    sparse_len = len(state.query_embedding.get("sparse_vector", [{}])[0].get("indices", [])) if state.query_embedding else 0
    print(f"🧩 [Embedder] Dense dim = {dense_dim}, Sparse tokens = {sparse_len}")

    # 6️⃣ Vector DB Search
    state = search_vector_db(state, top_k=5, alpha=0.7)
    print(f"🔍 [VectorDB] Found {len(state.search_results)} results | llm_status={state.llm_status}")

    # 7️⃣ Retriever
    state = retrieve_documents(state, max_chars=1500)
    print(f"📄 [Retriever] Retrieved {len(state.retrieved_docs)} docs | context_len={len(state.context)}")

    # 8️⃣ Build Prompt
    state = build_prompt(state, max_context_chars=1200)
    print(f"🧠 [Prompt] length={len(state.prompt)}, docs={len(state.retrieved_docs)}")

    # 9️⃣ In preview prompt
    print("\n=== 🔍 PROMPT SAMPLE (first 600 chars) ===")
    print(state.prompt[:600])
    print("---------------------------------------------")
    print(f"🧠 LLM status: {getattr(state, 'llm_status', 'undefined')}")

    print("\n✅ [Test] Hoàn tất kiểm thử PROMPT BUILDER.\n")


if __name__ == "__main__":
    test_prompt_realdata()



=== 🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT ===

🧠 [Processor] processed_query = hôm nay có tin tức gì mới
⏰ [TimeFilter] 2025-10-27 00:00:00+07:00 → 2025-10-27 23:59:59+07:00
➡️ [Router] route_to = rag, api_type = None
🧩 [Embedder] Dense dim = 384, Sparse tokens = 7
🔍 [VectorDB] Found 5 results | llm_status=vector_db_success
📄 [Retriever] Retrieved 2 docs | context_len=1761
🧠 [Prompt] length=2518, docs=2

=== 🔍 PROMPT SAMPLE (first 600 chars) ===
- Luôn trả lời bằng TIẾNG VIỆT tự nhiên.
- KHÔNG đưa ra khuyến nghị đầu tư.
- Chỉ xuất phần trả lời, đừng lặp lại hướng dẫn.

## Instruction:
Bạn là trợ lý AI tài chính Việt Nam, chuyên phân tích xu hướng thị trường, cổ phiếu và tin tức.
1. Khi intent = "market": KẾT HỢP dữ liệu API (giá cổ phiếu, thị trường) với tin tức.
2. Khi intent = "stock": Tóm tắt thông tin giá/hành vi cổ phiếu.
## Constraints:
- Trả lời bằng TIẾNG VIỆT, ngắn gọn, tự nhiên.
- KHÔNG in mã code, KHÔNG trả JSON thô.
- KHÔNG nói “Tôi không phải chuyên gia tài chính”.
- Gi

In [7]:
from modules.nodes.response_generator import response_node  # import file chứa response_node
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.router import route_intent
from modules.api import stock_api
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.nodes.prompt_builder import build_prompt

def _maybe_call_api(state: GlobalState):
    """
    Tùy thuộc vào api_type do router quyết định, gọi API tương ứng
    để có state.api_response trước khi vào RAG (nếu cần).
    Router mới đã tự set sẵn api_response cho hầu hết case.
    Ở đây ta chỉ 'bổ trợ' khi cần demo nhanh.
    """
    api_type = getattr(state, "api_type", None)
    # Stock quote / history đã được router set sẵn state.api_response.
    # Market summary cũng tương tự. Ở đây chỉ minh hoạ gọi thẳng 1 mã nếu cần.
    if api_type in ["stock_quote"] and not state.api_response:
        # nếu chưa có, format lại theo stock_api
        # lấy mã từ tickers nếu có
        sym = (state.tickers[0] if state.tickers else "VCB")
        state.api_response = stock_api.format_stock_info(sym)
    # stock_history đã có state.api_response từ router
    # market_analysis đã có state.api_response (tổng quan/thêm brief dự báo)
    # forecast đã có state.api_response (một dự báo duy nhất)
    # weather/time cũng đã có
    return state

def _run_pipeline_once(user_query: str, debug=True):
    print(f"\n=== 🧩 SCENARIO: {user_query} ===")
    state = GlobalState(user_query=user_query, debug=debug)

    # 1) Processor
    state = processor_query(state)
    print(f"🧠 [Processor] processed_query = {state.processed_query}")
    print(f"   intent={state.intent}, tickers={state.tickers}, time_filter={state.time_filter}")

    # 2) Router
    state = route_intent(state)
    print(f"🧭 [Router] route_to = {state.route_to}, api_type = {state.api_type}")

    # 3) (Optional) API – nếu là API route/hybrid có api_response sẵn
    state = _maybe_call_api(state)
    if getattr(state, "api_response", None):
        preview = state.api_response if len(state.api_response) < 300 else state.api_response[:300] + "..."
        print(f"📊 [API] response preview:\n{preview}")

    # 4) Embedder (tự skip nếu route_to không phải rag/hybrid)
    state = embed_query(state)
    dense_dim = len(state.query_embedding['dense_vector']) if state.query_embedding else 0
    print(f"🧩 [Embedder] status={state.llm_status}, dense_dim={dense_dim}")

    # 5) Vector DB (tự skip nếu không phải rag/hybrid)
    state = search_vector_db(state, top_k=8, alpha=0.7)
    print(f"🔍 [VectorDB] results={len(state.search_results)}, llm_status={state.llm_status}")

    # 6) Retriever (tự skip nếu không phải rag/hybrid)
    state = retrieve_documents(state, max_chars=1500)
    print(f"📄 [Retriever] docs={len(state.retrieved_docs)}, context_len={len(state.context)}")

    # 7) Prompt Builder (tự skip nếu không phải rag/hybrid)
    state = build_prompt(state, max_context_chars=2000)
    print(f"📜 [Prompt] status={state.llm_status}, length={len(state.prompt or '')}")

    # 8) Response (LLM) — nếu route_to != rag/hybrid thì response_node trả lại api_response
    state = response_node(state)
    print(f"🤖 [Response] route={state.route}, len={len(state.final_answer)}")
    print(f"🧾 [Final Answer]:\n{state.final_answer[:2000]}")

    # 9) Debug summary
    if getattr(state, "debug_info", None):
        print("\n=== 🧩 DEBUG SUMMARY ===")
        for k, v in state.debug_info.items():
            print(f"{k}: {v}")

    return state

def test_full_rag_pipeline_realdata():
    """
    🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT (END-TO-END)

    Bao phủ các intent:
      1) stock_quote:          "Giá VCB hôm nay?"
      2) stock_history:        "Cho mình lịch sử VCB 7 ngày"
      3) market_analysis:      "Phân tích cổ phiếu SHS hôm nay?"
      4) forecast-only:        "Dự đoán phiên tới của FPT"
      5) rag/news:             "Tin tức mới nhất về ngân hàng tuần này"
    """
    scenarios = [
        "Hôm nay có tin tức gì mới?",
    ]
    for q in scenarios:
        _run_pipeline_once(q, debug=True)

if __name__ == "__main__":
    test_full_rag_pipeline_realdata()



=== 🧩 SCENARIO: Hôm nay có tin tức gì mới? ===
🧠 [Processor] processed_query = hôm nay có tin tức gì mới
   intent=rag, tickers=[], time_filter=(1761498000, 1761584399)
🧭 [Router] route_to = rag, api_type = None
🧩 [Embedder] status=embed_success, dense_dim=384
🔍 [VectorDB] results=7, llm_status=vector_db_success
📄 [Retriever] docs=2, context_len=1761
📜 [Prompt] status=prompt_built_success, length=3077
🤖 [Response] route=rag, len=790
🧾 [Final Answer]:
📌 Một số diễn biến đáng chú ý gần đây:
1. Lịch sự kiện và tin vắn chứng khoán ngày 27/10/2025 (27-10-2025 05:00:00)
   tư và Xây dựng Thủy lợi Lâm Đồng: Kế toán trưởng, Người được ủy quyền công bố thông tin Nguyễn Thị Thu Hương không bán được CP nào trên tổng số 15.000 CP đăng ký bán. Số lượng cổ phiếu nắm giữ sau khi thực hiện giao dịch là 33.604 CP (tỷ lệ...
2. Có tin báo cáo tài chính doanh nghiệp, chứng khoán sẽ 'thoát hiểm'? (27-10-2025 00:07:00)
   vọng lợi nhuận quý III tích cực. Chúng tôi giữ quan điểm thận trọng, lựa chọn cơ hội 

In [1]:
from modules.core.graph import build_graph
from modules.core.state import GlobalState

graph = build_graph()

init_state = GlobalState(
    user_query="Hôm nay có tin tức gì mới?"
# "VCB biến động thế nào trong phiên?"
# "Cổ phiếu FPT tăng hay giảm hôm nay?"
)

state = graph.invoke(init_state)

# DEBUG Ở ĐÂY
print("==== DEBUG FINAL STATE ====")
print("route_to:", state.get("route_to"))
print("intent:", state.get("intent"))
print("llm_status:", state.get("llm_status"))
print("final_answer:", state.get("final_answer", "")[:5000], "\n")
api_resp = state.get("api_response")
if api_resp is None:
    api_resp_preview = "None"
else:
    api_resp_preview = str(api_resp)[:300]

print("api_response:", api_resp_preview, "\n")


docs = state.get("retrieved_docs", [])
print("retrieved_docs len:", len(docs))
if docs:
    print("doc[0] title:", docs[0].get("title"))
    print("doc[0] time:", docs[0].get("time"))
    print("doc[0] snippet:", (docs[0].get("content","")[:200]).strip(), "\n")

print("prompt preview:\n", state.get("prompt","")[:800])


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1027 13:56:28.814000 7760 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.
EmbedderServices device: cuda
RerankerServices device: cuda


Device set to use cuda:0
c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\vnai\scope\profile.py:562: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


==== DEBUG FINAL STATE ====
route_to: rag
intent: rag
llm_status: response_generated
final_answer: VNINDEX giảm 2,77% xuống 1,683,18 điểm, tuần thứ 14 liên tiếp khối ngoại bán ròng với tổng giá trị hơn 4.360 tỷ đồng. Một số giao dịch đáng chú ý: DMS (DMS) Phó Giám đốc Ngô Văn Phong bán hết 38.100 CP; TS3 (TS3) Cao Thị Thu Thủy mua thêm 4.600 CP; PMW đăng ký mua 299.608 CP từ 3-11 đến 28-11. CMP và PCC công bố ngày chốt cổ tức năm 2024 (tỷ lệ 1,49% và 15%). Thị trường tiếp tục trong xu hướng tích lũy, kiểm định vùng 1.700 điểm. 

api_response: None 

retrieved_docs len: 2
doc[0] title: Lịch sự kiện và tin vắn chứng khoán ngày 27/10/2025
doc[0] time: 27-10-2025 05:00:00
doc[0] snippet: tư và Xây dựng Thủy lợi Lâm Đồng: Kế toán trưởng, Người được ủy quyền công bố thông tin Nguyễn Thị Thu Hương không bán được CP nào trên tổng số 15.000 CP đăng ký bán. Số lượng cổ phiếu nắm giữ sau khi 

prompt preview:
 - Luôn trả lời bằng TIẾNG VIỆT tự nhiên.
- Chỉ xuất phần trả lời, đừng lặp lại hướng dẫ